In [1]:
## configuring and cloning into git

In [2]:
!git clone https://github.com/arathee2/solar-panel-detection.git

Cloning into 'solar-panel-detection'...
remote: Enumerating objects: 2174, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 2174 (delta 3), reused 10 (delta 2), pack-reused 2158 (from 1)
Receiving objects: 100% (2174/2174), 63.79 MiB | 15.76 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Updating files: 100% (2077/2077), done.


In [3]:
%cd solar-panel-detection


/content/solar-panel-detection


In [4]:
#setting git global vars
!git config --global user.email "aajinroyind@gmail.com"
!git config --global user.name "aajinr"

In [6]:
!git config --global --list


user.email=aajinroyind@gmail.com
user.name=aajinr


In [7]:
!git checkout -b yolov8


Switched to a new branch 'yolov8'


In [8]:
## importing libraries

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import precision_score, recall_score, classification_report

In [ ]:
## switching directories to read file

In [11]:
import os
print(os.getcwd())

/content/solar-panel-detection


In [12]:
%cd data

/content/solar-panel-detection/data


In [ ]:
# Load the Excel file
df = pd.read_csv('labels_training.csv')

In [ ]:

#converting columns to required data format
df['id'] = df['id'].astype(str)
df['label'] = df['label'].astype(int)

In [ ]:
print(df.head())
print(df.dtypes)


  id  label
0  0      0
1  1      0
2  2      1
3  3      1
4  4      1
id       object
label     int64
dtype: object


In [ ]:
#creating the custom dataset class
class SolarPanelDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Construct the path to the image file
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx, 0] + '.tif')
        print("Trying to open:", img_name)  # Print the file path before attempting to open
        image = Image.open(img_name)  # This might raise FileNotFoundError

        # Open the image file (PIL supports .tif files)
        image = Image.open(img_name).convert('RGB')  # Ensure image is in RGB mode

        # Get the label
        label = self.dataframe.iloc[idx, 1]

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label



In [ ]:
# Create Dataset and DataLoader
train_dataset = SolarPanelDataset(dataframe=df, image_dir='training', transform=transform)

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(train_dataset, test_size=0.2, random_state=42)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')


Streaming output truncated to the last 5000 lines.
Trying to open: training/672.tif
Trying to open: training/684.tif
Trying to open: training/75.tif
Trying to open: training/1192.tif
Trying to open: training/929.tif
Trying to open: training/331.tif
Trying to open: training/290.tif
Trying to open: training/209.tif
Trying to open: training/1330.tif
Trying to open: training/536.tif
Trying to open: training/216.tif
Trying to open: training/552.tif
Trying to open: training/161.tif
Trying to open: training/84.tif
Trying to open: training/1009.tif
Trying to open: training/744.tif
Trying to open: training/235.tif
Trying to open: training/678.tif
Trying to open: training/1062.tif
Trying to open: training/1375.tif
Trying to open: training/26.tif
Trying to open: training/988.tif
Trying to open: training/1112.tif
Trying to open: training/575.tif
Trying to open: training/1093.tif
Trying to open: training/320.tif
Trying to open: training/1301.tif
Trying to open: training/1274.tif
Trying to open: tra

In [ ]:
# Evaluation on validation set
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.numpy())
        all_preds.extend(preds.numpy())

Trying to open: testing/1116.tif


FileNotFoundError: [Errno 2] No such file or directory: 'testing/1116.tif'

In [ ]:
# Calculate metrics
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Classification Report:')
print(report)